In [1]:
import numpy as np 
import pandas as pd 
import pylab as plt
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
from scipy import integrate, optimize
import warnings
import seaborn as sns
plt.style.use("seaborn")
warnings.filterwarnings('ignore')

In [2]:
#Import dataset
input_dir = '../input/data-covid-harian/convertcsv.csv'
data = pd.read_csv(input_dir)
print(data.tail())

In [3]:
#Data Infected dan Recovered/Removed
data_IR = data[['dirawat_kumulatif']]
data_IR['rec'] = data['sembuh_kumulatif'] + data['meninggal_kumulatif']
data_IR = data_IR.rename(columns={"dirawat_kumulatif": "inf"})
data_IR.tail()

In [4]:
#Index prediksi dimulai (hari ke-berapa)
split_num = len(data_IR)-(len(data_IR)//8)

In [5]:
#Formula SIRS
# Susceptible equation
def fa(N, a, b, c, x, beta):
    fa = -(beta*a*b) / N + x * c
    return fa

# Infected equation
def fb(N, a, b, x, beta, gamma):
    fb = (beta*a*b) / N - gamma*b
    return fb

# Recovered/deceased equation
def fc(N, b, c, x, gamma):
    fc = gamma*b - x*c
    return fc

In [6]:
# Runge-Kutta method of 4rth order for 3 dimensions (susceptible a, infected b and recovered r)
def rK4(N, a, b, c, beta, gamma, x, hs):
    a1 = fa(N, a, b, c, x, beta)*hs
    b1 = fb(N, a, b, x, beta, gamma)*hs
    c1 = fc(N, b, c, x, gamma)*hs
    ak = a + a1*0.5
    bk = b + b1*0.5
    ck = c + c1*0.5
    a2 = fa(N, ak, bk, ck, x, beta)*hs
    b2 = fb(N, ak, bk, x, beta, gamma)*hs
    c2 = fc(N, bk, ck, x, gamma)*hs
    ak = a + a2*0.5
    bk = b + b2*0.5
    ck = c + c2*0.5
    a3 = fa(N, ak, bk, ck, x, beta)*hs
    b3 = fb(N, ak, bk, x, beta, gamma)*hs
    c3 = fc(N, bk, ck, x, gamma)*hs
    ak = a + a3
    bk = b + b3
    ck = c + c3
    a4 = fa(N, ak, bk, ck, x, beta)*hs
    b4 = fb(N, ak, bk, x, beta, gamma)*hs
    c4 = fc(N, bk, ck, x, gamma)*hs
    a = a + (a1 + 2*(a2 + a3) + a4)/6
    b = b + (b1 + 2*(b2 + b3) + b4)/6
    c = c + (c1 + 2*(c2 + c3) + c4)/6
    return a, b, c

In [7]:
N = 272229372
b0 = 0
beta = 0.12
gamma = 0.1
hs = 0.1
x = 0.01

"""
N = total number of population
beta = transition rate S->I
gamma = transition rate I->R
k =  denotes the constant degree distribution of the network (average value for networks in which 
the probability of finding a node with a different connectivity decays exponentially fast
hs = jump step of the numerical integration
"""

# Initial condition
"""
a = Susceptible
b = Infected
c = Removed
"""
b = float(data_IR.iloc[split_num]['inf'])
c = float(data_IR.iloc[split_num]['rec'])
a = float(N-b-c)

idx, sus, inf, rec= [],[],[],[]
for i in range(split_num,len(data_IR)): # Run for a certain number of time-steps
    idx.append(i)
    sus.append(a)
    inf.append(b)
    rec.append(c)
    a,b,c = rK4(N, a, b, c, beta, gamma, x, hs)


In [8]:
pred_IR = pd.DataFrame({'idx': idx, 'inf': inf, 'rec': rec})
pred_IR = pred_IR.set_index('idx')

In [9]:
f = plt.figure(figsize=(10,8)) 
plt.plot(data_IR.index,data_IR['inf'], 'r.', label='Infected');
plt.plot(data_IR.index,data_IR['rec'], 'c.', label='Removed');
plt.plot(pred_IR.index,pred_IR['inf'], 'm.', label='Pred infected');
plt.plot(pred_IR.index,pred_IR['rec'], 'b.', label='Pred removed');
plt.title("SIR model")
plt.xlabel("hari ke-", fontsize=10);
plt.ylabel("Populasi", fontsize=10);
plt.legend(loc='best')
plt.xlim(0,700)
plt.savefig('SIR_result.png')
plt.show()